# Top 10 Important Features - (Ames, Iowa Housing)

## Loading the Data and Initial Cleanup

In [1]:
cd ..

/home/jovyan/CSX_450_2_Project_Final


In [2]:
run __init__.py

In [3]:
%run src/load_data.py

In [4]:
from sklearn import linear_model
from sklearn.tree import DecisionTreeClassifier

from tqdm import tqdm 
from time import time

## One-hot Encoding of Categoraical Features

In [5]:
categorial_df = housing_df.select_dtypes(include=['category']).copy()

In [6]:
# Categorial Features Only
categorial_final_df = pd.get_dummies(categorial_df, prefix_sep='_',)

In [7]:
# All Features
final_features_df = pd.merge(categorial_final_df, numeric_log_sc_df, left_index=True, right_index=True)

In [8]:
# All Features  & Target
final_df = pd.merge(final_features_df, housing_df[['SalePrice']], left_index=True, right_index=True)

In [9]:
#!conda install tqdm --yes

## Feature Selection - Using Decision Tree

In [10]:
from sklearn.tree import DecisionTreeClassifier
# load data
data_array_X = final_features_df.values
data_array_Y = final_df[['SalePrice']]
X = data_array_X
Y = data_array_Y

#X = data_array[:,1:403]
#Y = data_array[:,404]
# feature extraction
model = DecisionTreeClassifier(max_features=200)
model.fit(X, Y)
#print(model.feature_importances_)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=200, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [11]:
feature_importances = model.feature_importances_
features = final_features_df.columns
feature_importances = pd.Series(feature_importances.T.ravel(), index=features)

In [12]:
sorted_importances = feature_importances.sort_values(ascending=False)
sorted_importances = pd.DataFrame(sorted_importances, columns=['importances'])
sorted_importances['importance_feature'] = sorted_importances.index
sorted_importances.reset_index(drop=True, inplace=True)
sorted_importances.head(10)

,importances,importance_feature
0,0.029739,TotalBsmtSF
1,0.028717,GrLivArea
2,0.027508,LotArea
3,0.027337,GarageArea
4,0.027336,BsmtUnfSF
5,0.025710,YearRemodAdd
6,0.024686,1stFlrSF
7,0.022363,LotFrontage
8,0.022163,YearBuilt
9,0.019868,WoodDeckSF


## Feature Selection - Using Lasso

In [13]:
vr_by_f1_test_scores = []
for feature in tqdm(final_features_df.columns):
    model = linear_model.Lasso()
    model.fit( final_features_df[[feature]],final_df[['SalePrice']])
    test_score = model.score(final_features_df[[feature]],final_df[['SalePrice']])
    if test_score > 0:
        vr_by_f1_test_scores.append({'feature': feature, 'score' : test_score})

100%|██████████| 404/404 [00:01<00:00, 311.85it/s]


In [14]:
vr_by_f1_results = pd.DataFrame(vr_by_f1_test_scores).sort_values('score', ascending=False)
vr_by_f1_results.head(10)

,feature,score
389,GrLivArea,0.501615
391,GarageArea,0.408527
385,TotalBsmtSF,0.402777
386,1stFlrSF,0.391249
316,GarageCars_3.0,0.385055
189,BsmtQual_Ex,0.350487
178,ExterQual_TA,0.348998
379,YearBuilt,0.311887
264,KitchenQual_Ex,0.286481
380,YearRemodAdd,0.283306


## Observation

As we can see, both lists (results from Decision Tree, and Lasso) include a common set of features along with some variations. We will continue our analysis using the features that were picked by the Decision Tree model. We may want to explore both (and even more) options to see which set will result in a better model.